In [179]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk
import numpy as np
import re
import sys
from nltk import NaiveBayesClassifier
import nltk.classify
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
import re
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
import pymorphy2
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [180]:
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, datasets
from sklearn.ensemble import GradientBoostingClassifier

In [181]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from functools import lru_cache
import pymorphy2

In [182]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    return text
@lru_cache(100000)
def lemma_tokenizer(text):
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [morph.parse(token)[0].normal_form for
                         token in tokens]
    return lemmatized_tokens
def ocenkaSex(s):
    if s == '+':
        return 100
    if s == '−':
        return 0
    return 50
def ocenka(s):
    if s == '?':
        return 0
    if s == 'Communication':
        return 1
    if s == 'Price':
        return 2
    if s == 'Quality':
        return 3
    if s == 'Safety':
        return 4
    assert(0)

In [183]:
path = 'train.csv'
c = pd.read_csv(path, encoding='UTF-8', error_bad_lines=False)

In [184]:
stopWords = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()
tf_transformer = TfidfTransformer()
vectorizer = CountVectorizer(stop_words=stopWords, preprocessor=preprocess_text, tokenizer=lemma_tokenizer)

In [185]:
stopWords = stopwords.words('russian')
stopWords.remove('не')
stopWords.remove('нет')

In [186]:
morph = pymorphy2.MorphAnalyzer()
sentences = c['sentence'].tolist()[0:15000]
ocenkiSex = list(map(ocenkaSex, c['sentiment'].tolist()))[0:15000]
ocenki = list(map(ocenka, c['1category'].tolist()))[0:15000]
X = vectorizer.fit_transform(sentences)
YSex = np.array(ocenkiSex)
Y = np.array(ocenki)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:382: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [187]:
X_train_tf = tf_transformer.fit_transform(X)

In [200]:
sents = pd.read_csv('test_for_participants.csv', encoding='UTF-8', error_bad_lines=False)['sentence'].tolist()
X_test = tf_transformer.transform(vectorizer.transform(sents))

In [189]:
model_RandomForestClassifier = RandomForestClassifier()
classifier = RandomForestClassifier()

In [197]:
classifier.fit(X_train_tf, np.array(ocenki))
model_RandomForestClassifier.fit(X_train_tf, YSex)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [214]:
import csv
import math
y_pred = classifier.predict(X_test)
y_pred_Sex = model_RandomForestClassifier.predict(X_test)
y_proba = classifier.predict_proba(X_test)[:, :]
y_probaSex = model_RandomForestClassifier.predict_proba(X_test)[:, :]

# l = []

with open('new.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['sentence', '1category', '1categoryprob', '2category', '2categoryprob', 'sentiment', 'sentimentprob']
    writer.writerow(header)
    for i in range(len(y_pred)):
        arr = []
        variants = ['?', 'Community', 'Price', 'Quality', 'Safety']
        variantsSex = ['-', '?', '+']
        classes = classifier.classes_
        top_classes = classes[np.argsort(-y_proba)[i][:2]]
        classesSex = model_RandomForestClassifier.classes_
        top_classesSex = classesSex[np.argsort(-y_probaSex)[i][:2]]
        arr.append(sents[i])
        arr.append(variants[top_classes[0]])
        big = max(y_proba[i][0], y_proba[i][1])
        bigSex = max(y_probaSex[i][0], y_probaSex[i][1])
        arr.append(f'{big:.2}')
        arr.append('')
        arr.append('')
        if (bigSex < 0.10):
            arr.append(variantsSex[classesSex[np.argsort(-y_probaSex)[i][2]]//50])
            arr.append(min(y_probaSex[i][0], y_probaSex[i][1], y_probaSex[i][2]))
        else:
            arr.append(variantsSex[top_classesSex[0]//50])
            arr.append(f'{bigSex:.2}')
        writer.writerow(arr)
        
# print(sum(l)/len(l))